## EDA

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

### Read Project Data

In [3]:
url = "https://raw.githubusercontent.com/MIDS-at-Duke/unifying-data-science-2023-project-team3/cdc_data/20_intermediate_files/project_data.csv?token=GHSAT0AAAAAACAZ7EJZSIGQ4MOQLVLJY2XEZCBQFKA"
data = pd.read_csv(url)
data.head()

,Unnamed: 0,CoC Number,Coc,Year,permanent_housing,homeless,poverty,unemployment_rate,Population,Male,...,White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,Two or More Races,Non Hispanic,Hispanic,Drug Deaths,Suicide Deaths
0,0,AZ-502,"Phoenix,Mesa/Maricopa",2007,2014.0,8448,3832171.0,3.2,60969,31121,...,49894,3613,2261,1937,166,3098,33167,27802,575.0,568.0
1,1,AZ-502,"Phoenix,Mesa/Maricopa",2008,2523.0,7189,3903475.0,5.1,59600,30420,...,48065,3766,2332,1985,146,3306,32312,27288,526.0,540.0
2,2,AZ-502,"Phoenix,Mesa/Maricopa",2009,2529.0,7889,3972331.0,8.9,52910,26919,...,42087,3550,1949,1909,148,3267,28793,24117,566.0,524.0
3,3,AZ-502,"Phoenix,Mesa/Maricopa",2010,3013.0,6999,3772257.0,9.5,54424,27741,...,43435,3559,2005,1982,149,3294,29386,25038,609.0,564.0
4,4,AZ-502,"Phoenix,Mesa/Maricopa",2011,3214.0,5831,3824961.0,8.5,3875371,1915659,...,3303462,212348,101821,149099,10314,98327,2724688,1150683,627.0,621.0
